In [2]:
from tools import get_puzzle, show_problem_1, show_problem_2

TODAY = 17
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)

https://adventofcode.com/2023/day/17
## --- Day 17: Clumsy Crucible ---


The lava starts flowing rapidly once the Lava Production Facility is operational. As youleave, the reindeer offers you a parachute, allowing you to quickly reach Gear Island.


As you descend, your bird's-eye view of Gear Island reveals why you had trouble finding anyone on your way up: half of Gear Island is empty, but the half below you is a giant factory city!


You land near the gradually-filling pool of lava at the base of your new **lavafall** . Lavaducts will eventually carry the lava throughout the city, but to make use of it immediately, Elves are loading it into large [crucibles](https://en.wikipedia.org/wiki/Crucible) on wheels.


The crucibles are top-heavy and pushed by hand. Unfortunately, the crucibles become very difficult to steer at high speeds, and so it can be hard to go in a straight line for very long.


To get Desert Island the machine parts it needs as soon as possible, you'll need to find the best way to get the crucible **from the lava pool to the machine parts factory** . To do this, you need to minimize **heat loss** while choosing a route that doesn't require the crucible to go in a **straight line** for too long.


Fortunately, the Elves here have a map (your puzzle input) that uses traffic patterns, ambient temperature, and hundreds of other parameters to calculate exactly how much heat loss can be expected for a crucible entering any particular city block.


For example:


```
 2413432311323
 3215453535623
 3255245654254
 3446585845452
 4546657867536
 1438598798454
 4457876987766
 3637877979653
 4654967986887
 4564679986453
 1224686865563
 2546548887735
 4322674655533

```


Each city block is marked by a single digit that represents the **amount of heat loss if the crucible enters that block** . The starting point, the lava pool, is the top-left city block; the destination, the machine parts factory, is the bottom-right city block. (Because you already start in the top-left block, you don't incur that block's heat loss unless you leave that block and then return to it.)


Because it is difficult to keep the top-heavy crucible going in a straight line for very long, it can move **at most three blocks** in a single direction before it must turn 90 degrees left or right. The crucible also can't reverse direction; after entering each city block, it may only turn left, continue straight, or turn right.


One way to **minimize heat loss** is this path:


>>^>>>v>>>vv>>vv>vvv>465496798688v456467998645v12246868655<v25465488877vv>```
 2341323
 32355623
 3255245654
 344658584552
 45466578676
 143859879844
 445787698776
 363787797965
 43226746555

```


This path never moves more than three consecutive blocks in the same direction and incurs a heat loss of only102``.


Directing the crucible from the lava pool to the machine parts factory, but not moving more than three consecutive blocks in the same direction, **what is the least heat loss it can incur?** 




In [44]:

class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
    
    def __repr__(self):
        return f"Pos: {self.position} f:{self.f} g:{self.g} h:{self.h}"


def astar(maze, start, end):
    """Returns a list of tuples as a path from the given start to the given end in the given maze"""

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:


        # Get the current node
        current_node = open_list[0]
        current_index = 0
        
        #for index, item in enumerate(open_list):
        #    if item.f < current_node.f:
        #        current_node = item
        #        current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        print(f"Closed: {len(open_list)}   Open:{len(open_list)} ")

        # Found the goal
        if current_node.position == end_node.position:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            print(f" f:{current_node.f} g:{current_node.g} h:{current_node.h}")
            print ( path[::-1]) # Return reversed path
            continue

        # Generate children
        children = []
        for direction in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Non-diagonal adjacent squares

            # Get node position
            

            new_position = validate_new_position(current_node, direction, maze)

            if new_position:
                # Make sure walkable terrain
                #if maze[node_position[0]][node_position[1]] != 0:
                    #continue

                # Create new node
                new_node = Node(current_node, new_position)

                # Append
                children.append(new_node)

        print (f"\tDOING { current_node.position}")

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
                
            print (f"child {maze[child.position[0]][child.position[1]]}")
            
            child.g = current_node.g + maze[child.position[0]][child.position[1]]
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child.position == open_node.position:
                    if child.f < open_node.f:
                        print(f"{open_node}  {child}")
                        open_node.f = child.f
                        open_node.g = child.g
                        open_node.h = child.h
                        open_node.parent = child.parent
                    continue

            # Add the child to the open list
            open_list.append(child)

def validate_new_position(current_node, direction, maze):

    new_position = (current_node.position[0] + direction[0], current_node.position[1] + direction[1])
    
    # Make sure within range
    if new_position[0] > (len(maze) - 1) or new_position[0] < 0 or new_position[1] > (len(maze[len(maze)-1]) -1) or new_position[1] < 0:
        return False
    # make sure not going backwards
    if current_node.parent and new_position[0] == current_node.parent.position[0]    and new_position[1] == current_node.parent.position[1]:
        return False
     # make sure this is not the 4th straight line move
    if current_node.parent and current_node.parent.parent and current_node.parent.parent.parent: 
        for i in [0,1]:
            if new_position[i] == current_node.position[i] == current_node.parent.position[i] == current_node.parent.parent.position[i] == current_node.parent.parent.parent.position[i]: 
                print("Not for times straight please!!")
                return False
    return new_position


def main(maze):

    start = (0, 0)
    end = (len(maze)-1, len(maze[0])-1)

    path = astar(maze, start, end)
    print(path)


if __name__ == '__main__':
    maze = [[int(c) for c in x] for x in puzzle.test]
    main(maze)

Closed: 0   Open:0 
	DOING (0, 0)
child 4
child 3
Closed: 1   Open:1 
	DOING (0, 1)
child 1
child 2
Closed: 2   Open:2 
	DOING (1, 0)
child 2
Pos: (1, 1) f:248 g:6 h:242  Pos: (1, 1) f:247 g:5 h:242
child 3
Closed: 3   Open:3 
	DOING (0, 2)
child 3
child 1
Closed: 4   Open:4 
	DOING (1, 1)
child 1
child 4
child 2
Closed: 6   Open:6 
	DOING (1, 1)
child 1
child 4
child 2
Closed: 8   Open:8 
	DOING (2, 0)
child 2
child 3
Closed: 9   Open:9 
Not for times straight please!!
	DOING (0, 3)
child 5
Closed: 9   Open:9 
	DOING (1, 2)
child 2
child 5
Pos: (1, 3) f:215 g:13 h:202  Pos: (1, 3) f:213 g:11 h:202
child 5
Closed: 11   Open:11 
	DOING (1, 2)
child 5
child 1
child 5
Closed: 13   Open:13 
	DOING (0, 1)
child 2
child 1
Closed: 14   Open:14 
	DOING (2, 1)
child 3
child 5
child 4
Closed: 16   Open:16 
	DOING (1, 2)
child 5
child 1
Pos: (0, 2) f:254 g:10 h:244  Pos: (0, 2) f:251 g:7 h:244
child 5
Pos: (2, 2) f:212 g:12 h:200  Pos: (2, 2) f:211 g:11 h:200
Closed: 18   Open:18 
	DOING (0, 1)
c

KeyboardInterrupt: 

In [11]:
i = ["2738", "7696"]
print ([[int(c) for c in x] for x in puzzle.test])

[[2, 4, 1, 3, 4, 3, 2, 3, 1, 1, 3, 2, 3], [3, 2, 1, 5, 4, 5, 3, 5, 3, 5, 6, 2, 3], [3, 2, 5, 5, 2, 4, 5, 6, 5, 4, 2, 5, 4], [3, 4, 4, 6, 5, 8, 5, 8, 4, 5, 4, 5, 2], [4, 5, 4, 6, 6, 5, 7, 8, 6, 7, 5, 3, 6], [1, 4, 3, 8, 5, 9, 8, 7, 9, 8, 4, 5, 4], [4, 4, 5, 7, 8, 7, 6, 9, 8, 7, 7, 6, 6], [3, 6, 3, 7, 8, 7, 7, 9, 7, 9, 6, 5, 3], [4, 6, 5, 4, 9, 6, 7, 9, 8, 6, 8, 8, 7], [4, 5, 6, 4, 6, 7, 9, 9, 8, 6, 4, 5, 3], [1, 2, 2, 4, 6, 8, 6, 8, 6, 5, 5, 6, 3], [2, 5, 4, 6, 5, 4, 8, 8, 8, 7, 7, 3, 5], [4, 3, 2, 2, 6, 7, 4, 6, 5, 5, 5, 3, 3]]


In [31]:
a=1
b=1
c=2
if a and b and c:
    print ('yes')

yes


In [29]:
a and b and c

2